<a href="https://colab.research.google.com/github/genie0320/langchain/blob/main/04_RAG_advance_parentDOC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 개요

RAG는 여러 과정을 거치면서 진행된다. 매 과정이 치명적으로 중요하다.

- 멀티쿼리 : 대충질문해도 좋은 답변을 원하는 이들을 위해 다음에 집중해야 한다.
- 페어런트 도큐먼트 : 앞뒤 문장을 잘 담아야 하고(chunk의 중요성)
- 셀프쿼리(질문재해석) : 시맨틱검색 말고 쿼리가 필요한 경우
  - 시맨틱검색이란 질문문장과 임베딩데이터의 벡터값에 따라 유사데이터를 걸러내는 것인데, 이 경우 질문의 모양이 조금만 달라져도 추출 데이터 자체가 달라진다. 이것은 '질문이 우선되는 구조'이기 때문이다.
  이 경우, 데이터를 중심으로 사용자의 질문을 참고하여 쿼리를 날려서 정리해야 할 필요가 생길 수 있다. 이것도 고려해야 한다.
- 타임 웨이티드 : 오래된 자료는 덜 참고했으면...
  - 최근에 올라간 자료에 무게를 둬서 답변을 참고했으면 좋겠다.

## 멀티쿼리 리트리버

사용자의 질문을 여러개의 유사질문으로 재생성하고, 각 질문에 대한 데이터를 추출해서 답변을 생성하는 방법.

# 코드

## Setting

In [1]:
# colab 환경설정
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['HUGGINGFACEHUB_API_TOKEN'] = userdata.get('HUGGINGFACEHUB_API_TOKEN')
os.environ['HF_TOKEN'] = userdata.get('HUGGINGFACEHUB_API_TOKEN')
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# !pip install -U -q langchain pypdf sentence_transformers chromadb langchain-openai
!pip install -q langchain pypdf sentence_transformers chromadb langchain-openai

In [3]:
# 유틸들
import math
import time

def trace(func):
    def wrapper():
        start = time.time()
        func()
        end = time.time()
        print(f"{end - start:.5f} sec")
    return wrapper

In [4]:
# ___ setting ___
LLM_MODEL = "gpt-3.5-turbo"
MAX = 265
TEMP = 1.5
# _______________

from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI

openai_compbot = OpenAI(
    temperature=TEMP,
    max_tokens = MAX,
    verbose = True,
)
openai_chatbot = ChatOpenAI(
    model_name = LLM_MODEL,
    temperature=TEMP,
    max_tokens = MAX,
    verbose = True,
)

In [5]:
URL = 'https://n.news.naver.com/article/002/0002319323?cds=news_media_pc&type=editn'
URLs = [
    URL,
    'https://n.news.naver.com/article/079/0003862456?cds=news_media_pc&type=editn'
]
SOURCE_folder = "/content/drive/MyDrive/data/test"
SOURCE = "/content/source/1-1그로스 해킹, 마케팅과 어떻게 다른가요_ - PUBLY.pdf"

chunk_size = 200

DB_URL = '/content/drive/MyDrive/vector_upgrade/'
MODEL_EMBED= "jhgan/ko-sbert-nli"

llm = openai_compbot
llm_chat = openai_chatbot

# ParentDocumentRetriever

사용자 질문에 맞는 청크를 반환받고 나서, 해당 청크의 부모가 된 원본을 다시 한번 참고한다는 흐름.

사용하는 이유는...
임베딩의 특성때문이다.

청크 하나당 임베딩값 한개가 만들어진다.
따라서 청크가 너무 크면, '너무 많은 것을 담아서 다른 임베딩데이터와 구분점을 찾기가 힘든' 흐릿한 임베딩이 될 가능성이 있다.

반면, 청크가 너무 작으면, '선명하긴 한데, 앞뒤 맥락이 없어서 아무짝에도 쓸모가 없는', 즉 LLM에게 제대로 된 커닝페이퍼가 되어줄 수 없는 context를 전달하게 된다.

따라서... 자식청크는 작게 만들어서 일종의 '목차'처럼 명확하게 이슈를 구분해주고, 해당 이슈에 대한 사용자의 질문이 오면, 그 이슈와 관련된 '엄마' 청크를 찾아서 context가 풍부한 자료를 전달할 수 있도록 하기 위해서다.

또한 엄마청크가 너무 크면 llm의 window error 가능성이 있으므로, 엄마청크도 적절한 크기로 잘라서 저장해둘 수 있다.

- [x] 그럼 속도가 느려지지 않는가? > 조온나 느리다. 진짜... GPU써도 너무너무 느리다.
- [x] 아예 처음부터 부모문서를 다 줘버리지 그래. > 임베딩의 특성상 엄마문서를 다 주면 윈도우크기를 벗어날 수도 있고, 임베딩 데이터간의 특성을 잃게 된다. 자식들은 다 개성이 뚜렷하지만, 엄마들은 다 ㅈㄹ같다는 부분에서 동일하다는 걸 생각해보자.

**InMemoryStore**
청크가 누구 자식인지를 기억해 놓는 기능.
- [x] 어차피 메타데이터가 붙지 않아? > loader에서 읽혀진 docs는 기본 메타데이터를 갖는다. 그리고 자식이건 엄마건 그 docs를 잘라서 마련된다. (단지, 작게 잘린놈, 덜 잘린놈으로 구분되는 것) 그런데... 어느집안 자식인지는 기본 메타로 알 수 있지만, 그게 어느엄마 자식인지는 따로 연결고리를 만들어줘야 한다.
알아보기 싫어서 예상만 하건대... 아마도, 먼저 엄마청크를 자르고, 그 청크를 작게 만들어서 자식청크를 만든다음, 엄마 이름을 추가 메타로 붙여주는 구조가 아닐까 싶다.

In [6]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

In [7]:
# 파일 로딩가능성 판단.
# file_path = '/content/drive/MyDrive/data/test/2023_디지털_분야_트렌드-게시_3.pdf'
# file_loader = PyPDFLoader(file_path)
# pages = file_loader.load()
# chracter_count = len(pages[0].page_content)

# if chracter_count == 50:
#     print(f'fail to load : {file_path}')

In [8]:
(
    '''
    # 왜 쓰는지 모르겠는 방법.
    > 텍스트로더의 경우, 문서를 하나의 객체로 반환한다.
    하지만 pdf로더의 경우, 한 페이지를 하나의 객체로 반환한다.
    따라서 청크 사이즈가 충분히 크다면(한 페이지를 다 들여올 수 있을만큼), 사실 Parent~는 쓸모가 없...을 것 같다.

    다만, 어차피 parent~ 에서도, 뒤에 부모문서의 크기가 너무 큰 경우,
    이를 split 해주는 기능을 넣고 있으므로...

    만약 들여오는 문서가 text라면 이 기능을 활용해볼만 하나,
    - pdf의 경우 한 페이지당 하나의 객체를 반환한다는 점.
    - directory로더를 사용하면 어차피 한 폴더 내의 문서들을 통째로 하나의 리스트로 담아준다는 점..
    에서 이 방식은 의미없는 방식이라고 할 수 있겠다.

    즉, 이 방식은, text 문서를 여러개 (왜 때문인지 따로) 가져와서
    다운스트림에서 사용할 수 있는 하나의 document 목록에 뭉쳐주기 위해서 이용한 고육지책이라고 할 수 있다.
    좀 현실적인 예를 보여줄 것이지... 왜 일케 예시를 위한 예시를 보여준 것인지 당췌 이해가...

    loaders = [
        TextLoader("../../paul_graham_essay.txt"),
        TextLoader("../../state_of_the_union.txt"),
    ]
    docs = []
    for loader in loaders:
        docs.extend(loader.load())

    len(docs) # 54
    '''
    )

# 재료준비
loader = DirectoryLoader(SOURCE_folder, glob='**/*.pdf', show_progress=True, loader_cls=PyPDFLoader, use_multithreading=True)
documents = loader.load_and_split()

print(len(documents), '\n\n', documents[0])

100%|██████████| 2/2 [00:05<00:00,  2.54s/it]

35 

 page_content='전망\n30  | 산은조사월보     ·런던금속거래소 구리 가격(달러/톤):8,365(‘22.12월)→9,004(’23.3 월)→8,322(6월)→8,029(10월)\n     ·런던금속거래소 니켈 가격(달러/톤):29,886(‘22.12 월)→23,651(’23.3 월)→20,346(6월)\n→17,903(10 월)\n❍ 2024년 철강 및 금속가격은 글로벌 수요 둔화와 공급 확대에 따른 수급 불균형 \n으로 하락 전망\n-중국 정부의 부동산 부양책 35)에도 불구하고 부동산경기 침체와 인프라 투자 \n둔화가 지속될 것으로 예상되는 가운데,미국 경기둔화로 철강 및 구리 등 금속\n가격은 하락세 유지\n-다만,주요 광물 생산국의 공급 확대*에 따른 가격 하방 압력에도 선진국을 \n중심으로 한 탈탄소화 흐름이 지속됨에 따라 배터리 소재 등 관련 금속(니켈·\n코발트 등 )수요 증가가 가격 하락을 제한\n    * 주요 생산지인 인도네시아와 필리핀 등 동남아 , 폐루 등 남미, 콩고민주공화국 등에서 \n알루미늄 , 니켈, 구리, 아연 등 금속 생산이 지속적으로 늘어날 전망\n     ·미국의 ‘인플레이션 감축법(IRA:Inflation Reduction Act)36)’시행,유럽의 \n‘RecHFT4’ocHFTt8ecHFTT6cHFT5/cHFT8’ 계획37)’등이 관련 금속인 주석,구리,니켈,아연 등의 수요 \n증가를 뒷받침\n35)중국 정부는 2023년‘생애 첫 주택구매자 요건’및‘주택구매제한’ 완화,‘주택담보대출 금리’인하 등 부동산 \n경기부양책을 발표\n36)동 법은 2022년 8월 16일 발효된 법으로 북미에서 조립되고 배터리 소재 및 부품을 미국 및 \nFTA를 맺은 국가에서 일정 비율 이상 조달한 전기차에만 보조금을 지급하는 내용을 포함하고 \n있으며,2030년까지 미국의 신차 판매량 중에서 전기차 비율을 50%목표\n37)러시아 화석연료에 대한 의존도를 빠르게 줄이고 녹색 전환을 가속화 하기 위한 계획.특히,\n20

In [9]:
ko_embed = HuggingFaceEmbeddings(
    model_name= MODEL_EMBED,
    model_kwargs={'device' : 'cpu'},
    encode_kwargs={'normalize_embeddings':True}
    )

# 자식구성
child_splitter = RecursiveCharacterTextSplitter(chunk_size=300)

vectordb = Chroma(
    collection_name = 'full_documents', # 여기 이름이 왜 child_documents가 아니라 full 인지 도무지 이해가 안됨.
    embedding_function=ko_embed
)

# 엄마구성
# The storage layer
store = InMemoryStore()

parent_splitter = RecursiveCharacterTextSplitter(chunk_size=1000) # 사실 엄마 스플리터는 필요가 없을 수도 있다. 어차피 PDF에서는 페이지당 하나의 doc객체를 반환하기 때문이다.

retriever = ParentDocumentRetriever(
    vectorstore = vectordb, # 저장소는 여기고
    docstore = store, # 임시 저장소는 여기인데,
    parent_splitter = parent_splitter, # 엄마는 이렇게 자르고
    child_splitter = child_splitter, # 애는 이렇게 처리해.
)

In [10]:
# 이제 위의 리트리버를 이용해서 로딩한 문서를 넣어준다.
# 다른 경우와는 달리, 위에서 설정했던 스플리터를 사용해서 얘가 다 알아서 잘라주고, 부모-자식간의 연결고리도 마련한다.
# 그래서 시간이 현기증날만큼 조온나 오래걸린다. GPU를 써도 오래걸린다. 와 짜증 진짜...

@trace
def make_context():
    retriever.add_documents(documents, ids=None)

make_context()
len(list(store.yield_keys()))

165.58265 sec


71

In [11]:
# 그냥 벡터DB에서 평소처럼 관련문서를 호출했을 때. 하나의 문서길이는 짧다.(자식만 불러왔을 테니까.)

query = "2024년 한국의 수출전망은 어떠한가?"

sub_docs = vectordb.similarity_search(query)



67

In [17]:
lens = []
for d in sub_docs:
    lens.append(len(d.page_content))

print(lens)

[67, 275, 283, 279]


In [12]:
# 그러나 이번에 만든 parent~ 리트리버를 사용해서 문서를 가져왔을 때는 문서 하나의 길이는 훨씬 길다는 걸 알 수 있다.
# 자식을 끄나풀 삼아 엄마를 잡아왔기때문.

parent_docs = retriever.get_relevant_documents(query)
len(parent_docs[0].page_content)

lens = []
for d in sub_docs:
    lens.append(len(d.page_content))

print(lens)

67

In [18]:
lens = []
for d in parent_docs:
    lens.append(len(d.page_content))

print(lens)

[67, 552, 924, 966]


- [ ] 근데 부모는 덩치가 큰데... 많은 부모가 잡혀왔을 때는 이걸 어떻게 context로 전달해줄 수 있나?
- [ ] 나는 왜 항상 내가 정했던 chunk 사이즈보다 크게 잘라지는거지...?

In [14]:
# import logging

# logging.basicConfig()
# logging.getLogger('langchain.retrievers.multi_query').setLevel(logging.INFO)

In [15]:
# unique_docs = retriever_from_llm.get_relevant_documents(query=question)
# len(unique_docs)

In [16]:
# unique_docs